# Opcional

1. Marcar el punto de giro con el ratón

2. Trasladar la imagen arrastrando con el ratón

In [8]:
import cv2
import numpy as np

clone = cv2.imread("images/cats.png")  # tu imagen
if clone is None:
    raise FileNotFoundError("No se pudo cargar la imagen.")

dragging = False
start_x, start_y = 0, 0
tx, ty = 0, 0

def mouse_drag(event, x, y, flags, param):
    global dragging, start_x, start_y, tx, ty
    if event == cv2.EVENT_LBUTTONDOWN:
        dragging = True
        start_x, start_y = x, y
    elif event == cv2.EVENT_MOUSEMOVE and dragging:
        tx, ty = x - start_x, y - start_y
    elif event == cv2.EVENT_LBUTTONUP:
        dragging = False

cv2.namedWindow("Traslación con ratón")
cv2.setMouseCallback("Traslación con ratón", mouse_drag)

while True:
    # Aplicar traslación
    M = np.float32([[1, 0, tx], [0, 1, ty]])
    moved = cv2.warpAffine(clone, M, (clone.shape[1], clone.shape[0]))
    cv2.imshow("Traslación con ratón", moved)

    key = cv2.waitKey(20) & 0xFF
    if key == ord("q"):
        # Guardar la última imagen antes de salir
        cv2.imwrite("output/imagen_trasladada.png", moved)
        print("Imagen guardada como 'output/imagen_trasladada.png'")
        break

cv2.destroyAllWindows()


Imagen guardada como 'output/imagen_trasladada.png'


3. Hacer la parte obligatoria sobre vídeo

In [14]:
"""
Video Transformations (Interactive)
- Run: python3 video_transform.py
- Requirements: opencv-python, numpy
"""

import cv2
import numpy as np

# ---------------- Config ----------------
VIDEO_SOURCE = 0   # 0 = webcam, or "video.mp4"
# VIDEO_SOURCE = "videos/Stairs.mp4"   # 0 = webcam, or "video.mp4"
WIN_NAME = "Video Transform (1:Affine  2:Projection  3:Distortion) - q:quit"
# ----------------------------------------

mode = 1  # 1=Affine (translate/rotate/scale), 2=Projection, 3=Distortion
center = None   # centro de rotación (x,y)
src_points = []   # para proyección (4 puntos de la imagen origen)
dst_points = []   # para proyección (4 puntos destino)
homography_ready = False
H = None

# ---------- utilidades ----------
def nothing(x):
    pass

def ordenar_cuatro_puntos(pts):
    pts = np.array(pts, dtype=np.float32)
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]
    return np.array([tl, tr, br, bl], dtype=np.float32)

def build_3x3_transform(center, angle_deg, sx, sy, tx, ty):
    # Construye matriz 3x3: T(tx,ty) * Trans(center) * R(angle) * Scale(sx,sy) * Trans(-center)
    a = np.deg2rad(angle_deg)
    ca, sa = np.cos(a), np.sin(a)
    # Scale * Rotate
    SR = np.array([
        [sx * ca, -sy * sa, 0.0],
        [sx * sa,  sy * ca, 0.0],
        [0.0,     0.0,     1.0]
    ], dtype=np.float32)
    cx, cy = center
    T1 = np.array([[1,0,cx],[0,1,cy],[0,0,1]], dtype=np.float32)
    T2 = np.array([[1,0,-cx],[0,1,-cy],[0,0,1]], dtype=np.float32)
    TT = np.array([[1,0,tx],[0,1,ty],[0,0,1]], dtype=np.float32)
    M = TT @ T1 @ SR @ T2
    return M

# ---------- mouse callback ----------
def on_mouse(event, x, y, flags, param):
    global center, src_points, dst_points, homography_ready, H, mode, first_frame_for_points

    if mode == 1:
        # en modo Affine: click izquierdo fija centro de rotación
        if event == cv2.EVENT_LBUTTONDOWN:
            center = (x, y)
            print(f"[Affine] Nuevo centro de rotación: {center}")
    elif mode == 2:
        # Modo Proyección: seleccionar src (primero) luego dst (luego)
        if event == cv2.EVENT_LBUTTONDOWN:
            if len(src_points) < 4:
                src_points.append([x, y])
                print(f"[Projection] src {len(src_points)}/4 -> {(x,y)}")
            elif len(dst_points) < 4:
                dst_points.append([x, y])
                print(f"[Projection] dst {len(dst_points)}/4 -> {(x,y)}")
            if len(src_points) == 4 and len(dst_points) == 4:
                # ordenar y calcular homografía
                src_o = ordenar_cuatro_puntos(src_points)
                dst_o = ordenar_cuatro_puntos(dst_points)
                H, _ = cv2.findHomography(src_o, dst_o)
                homography_ready = True
                print("[Projection] Homografía calculada. Se aplicará a todos los frames.")
    # modo 3 no usa mouse

# ---------- inicializar captura y ventanas ----------
cap = cv2.VideoCapture(VIDEO_SOURCE)
if not cap.isOpened():
    raise RuntimeError("No se pudo abrir la fuente de vídeo.")

ret, frame = cap.read()
if not ret:
    raise RuntimeError("No se pudo leer el primer frame.")
h, w = frame.shape[:2]

cv2.namedWindow(WIN_NAME, cv2.WINDOW_AUTOSIZE)
cv2.resizeWindow(WIN_NAME, min(1280,w), min(720,h))
cv2.setMouseCallback(WIN_NAME, on_mouse)

# ---------- trackbars ----------
# Affine trackbars
cv2.createTrackbar("tx", WIN_NAME, w//2, w, nothing)   # desplazamiento en x (se centrará en 0 con offset)
cv2.createTrackbar("ty", WIN_NAME, h//2, h, nothing)   # desplazamiento en y
cv2.createTrackbar("angle", WIN_NAME, 0, 360, nothing) # angulo 0..360
cv2.createTrackbar("scaleX*100", WIN_NAME, 100, 500, nothing) # scaleX (por 100)
cv2.createTrackbar("scaleY*100", WIN_NAME, 100, 500, nothing) # scaleY

# Distortion trackbars
cv2.createTrackbar("k1*1000", WIN_NAME, 0, 2000, nothing)
cv2.createTrackbar("k2*1000", WIN_NAME, 0, 2000, nothing)
cv2.createTrackbar("p1*1000", WIN_NAME, 1000, 2000, nothing)  # offset 1000 => 0
cv2.createTrackbar("p2*1000", WIN_NAME, 1000, 2000, nothing)
cv2.createTrackbar("k3*1000", WIN_NAME, 0, 2000, nothing)

print(__doc__)
print("Controles:")
print("  1 - Modo Affine (trackbars: tx,ty,angle,scaleX,scaleY). Clic izq fija centro de rotación.")
print("  2 - Modo Proyección (clic -> selecciona 4 src, luego 4 dst).")
print("  3 - Modo Distorsión (trackbars k1,k2,p1,p2,k3).")
print("  c - limpiar puntos de proyección")
print("  q - salir")

# ---------- precompute grid for distortion mapping ----------
# grid of pixel coordinates
u = np.arange(w)
v = np.arange(h)
uu, vv = np.meshgrid(u, v)
# normalized coords placeholders will be used in vectorized operations

# ---------- main loop ----------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    display = frame.copy()
    key = cv2.waitKey(1) & 0xFF

    # cambiar modo con teclas 1,2,3
    if key == ord('1'):
        mode = 1
        print("Modo: Affine")
    elif key == ord('2'):
        mode = 2
        print("Modo: Projection (clics para seleccionar 4 src / 4 dst)")
    elif key == ord('3'):
        mode = 3
        print("Modo: Distortion")
    elif key == ord('c'):
        src_points = []; dst_points = []; homography_ready = False; H = None
        print("Puntos de proyección limpiados.")
    elif key == ord('q'):
        break

    # Leer valores de trackbars (affine)
    tx_tb = cv2.getTrackbarPos("tx", WIN_NAME) - (w//2)   # centramos el trackbar en 0
    ty_tb = cv2.getTrackbarPos("ty", WIN_NAME) - (h//2)
    angle = cv2.getTrackbarPos("angle", WIN_NAME)
    sx = cv2.getTrackbarPos("scaleX*100", WIN_NAME) / 100.0
    sy = cv2.getTrackbarPos("scaleY*100", WIN_NAME) / 100.0

    # Distortion coefficients from trackbars (scaled)
    k1 = (cv2.getTrackbarPos("k1*1000", WIN_NAME) - 1000) / 1000.0 if cv2.getTrackbarPos("k1*1000", WIN_NAME) >= 1000 else cv2.getTrackbarPos("k1*1000", WIN_NAME)/1000.0
    k2 = (cv2.getTrackbarPos("k2*1000", WIN_NAME) - 1000) / 1000.0 if cv2.getTrackbarPos("k2*1000", WIN_NAME) >= 1000 else cv2.getTrackbarPos("k2*1000", WIN_NAME)/1000.0
    p1 = (cv2.getTrackbarPos("p1*1000", WIN_NAME) - 1000) / 1000.0
    p2 = (cv2.getTrackbarPos("p2*1000", WIN_NAME) - 1000) / 1000.0
    k3 = (cv2.getTrackbarPos("k3*1000", WIN_NAME) - 1000) / 1000.0 if cv2.getTrackbarPos("k3*1000", WIN_NAME) >= 1000 else cv2.getTrackbarPos("k3*1000", WIN_NAME)/1000.0
    # NOTE: we used trackbars 0..2000 to allow positive/negative easily (middle ~ offset). You can adjust ranges.

    # ---------------- Mode 1: Affine (translate/rotate/scale) ----------------
    if mode == 1:
        # si no se ha fijado centro, usa centro del frame
        if center is None:
            center = (w//2, h//2)
        M3 = build_3x3_transform(center, angle, sx, sy, tx_tb, ty_tb)
        # aplicar como perspectiva para usar 3x3
        warped = cv2.warpPerspective(frame, M3, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0))
        display = warped

        # dibujar centro y ejes guía
        cv2.circle(display, (int(center[0]), int(center[1])), 6, (0,255,0), -1)
        cv2.putText(display, f"Angle:{angle} sx:{sx:.2f} sy:{sy:.2f} tx:{tx_tb} ty:{ty_tb}", (10,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

    # ---------------- Mode 2: Projection ----------------
    elif mode == 2:
        # mostrar puntos actuales en el frame
        vis = frame.copy()
        for p in src_points:
            cv2.circle(vis, tuple(map(int,p)), 5, (0,255,0), -1)  # src = verde
        for p in dst_points:
            cv2.circle(vis, tuple(map(int,p)), 5, (0,0,255), -1)  # dst = rojo
        # dibujar poligonos si 4 seleccionados
        if len(src_points) == 4:
            cv2.polylines(vis, [np.int32(ordenar_cuatro_puntos(src_points))], True, (0,255,0), 2)
        if len(dst_points) == 4:
            cv2.polylines(vis, [np.int32(ordenar_cuatro_puntos(dst_points))], True, (0,0,255), 2)

        if homography_ready and H is not None:
            # aplicar homografía: mapea la imagen completa a la ventana de destino; fondo negro
            warped_full = cv2.warpPerspective(frame, H, (w, h))
            mask = np.zeros((h, w), dtype=np.uint8)
            cv2.fillPoly(mask, [np.int32(ordenar_cuatro_puntos(dst_points))], 255)
            result = np.zeros_like(frame)
            cv2.copyTo(warped_full, mask, result)
            display = result
            cv2.putText(display, "Projection applied (homography)", (10,30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        else:
            display = vis
            cv2.putText(display, "Click 4 src (verde) then 4 dst (red). Press 'c' to clear.", (10,30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

    # ---------------- Mode 3: Distortion ----------------
    elif mode == 3:
        # Simular distorsión usando mapeo directo (vectorizado)
        # parámetros de cámara aproximados
        fx = fy = 0.9 * w   # heurístico (ajustable)
        cx = w / 2.0
        cy = h / 2.0

        # Normalized coords
        x = (uu - cx) / fx
        y = (vv - cy) / fy
        r2 = x*x + y*y
        r4 = r2*r2
        r6 = r4*r2

        # Distorsión radial + tangencial
        radial = 1.0 + k1 * r2 + k2 * r4 + k3 * r6
        x_dist = x * radial + 2*p1*x*y + p2*(r2 + 2*x*x)
        y_dist = y * radial + p1*(r2 + 2*y*y) + 2*p2*x*y

        map_x = (x_dist * fx + cx).astype(np.float32)
        map_y = (y_dist * fy + cy).astype(np.float32)

        distorted = cv2.remap(frame, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
        display = distorted
        cv2.putText(display, f"k1:{k1:.3f} k2:{k2:.3f} p1:{p1:.3f} p2:{p2:.3f} k3:{k3:.3f}", (10,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

    # ---------- mostrar ----------
    cv2.imshow(WIN_NAME, display)

# ---------- cleanup ----------
cap.release()
cv2.destroyAllWindows()



Video Transformations (Interactive)
- Run: python3 video_transform.py
- Requirements: opencv-python, numpy

Controles:
  1 - Modo Affine (trackbars: tx,ty,angle,scaleX,scaleY). Clic izq fija centro de rotación.
  2 - Modo Proyección (clic -> selecciona 4 src, luego 4 dst).
  3 - Modo Distorsión (trackbars k1,k2,p1,p2,k3).
  c - limpiar puntos de proyección
  q - salir
[Affine] Nuevo centro de rotación: (161, 128)
[Affine] Nuevo centro de rotación: (488, 280)
[Affine] Nuevo centro de rotación: (265, 357)
[Affine] Nuevo centro de rotación: (194, 215)
[Affine] Nuevo centro de rotación: (389, 213)
[Affine] Nuevo centro de rotación: (411, 221)
[Affine] Nuevo centro de rotación: (259, 221)
[Affine] Nuevo centro de rotación: (509, 296)
[Affine] Nuevo centro de rotación: (360, 239)
[Affine] Nuevo centro de rotación: (383, 181)
[Affine] Nuevo centro de rotación: (486, 245)
[Affine] Nuevo centro de rotación: (334, 306)
[Affine] Nuevo centro de rotación: (219, 261)
[Affine] Nuevo centro de rotac

4. Transformación afín con 3 puntos

In [6]:
import cv2
import numpy as np

# --- Variables globales ---
src_points = []
dst_points = []
phase = 0   # 0 = seleccionando src, 1 = seleccionando dst
image = None
clone = None

def mouse_callback(event, x, y, flags, param):
    global src_points, dst_points, phase, image, clone

    if event == cv2.EVENT_LBUTTONDOWN:
        if phase == 0 and len(src_points) < 3:
            src_points.append([x, y])
            cv2.circle(image, (x, y), 5, (0, 255, 0), -1)  # verde
            cv2.imshow("Transformación Afín", image)

            if len(src_points) == 3:
                print("Selecciona ahora los 3 puntos destino (rojo).")
                phase = 1

        elif phase == 1 and len(dst_points) < 3:
            dst_points.append([x, y])
            cv2.circle(image, (x, y), 5, (0, 0, 255), -1)  # rojo
            cv2.imshow("Transformación Afín", image)

            if len(dst_points) == 3:
                print("Calculando transformación afín...")
                aplicar_transformacion()

def aplicar_transformacion():
    global src_points, dst_points, clone, image

    src = np.array(src_points, dtype=np.float32)
    dst = np.array(dst_points, dtype=np.float32)

    # matriz afín 2x3
    M = cv2.getAffineTransform(src, dst)

    h, w = clone.shape[:2]
    warped = cv2.warpAffine(clone, M, (w, h))

    cv2.imshow("Transformación Afín", warped)
    cv2.imwrite("output/resultado_affine.png", warped)
    print("Resultado guardado en 'resultado_affine.png'")

def main():
    global image, clone

    ruta = "images/cats.png"   # cambia por tu imagen
    clone = cv2.imread(ruta)
    if clone is None:
        raise FileNotFoundError("No se pudo cargar la imagen.")
    image = clone.copy()

    cv2.namedWindow("Transformación Afín")
    cv2.setMouseCallback("Transformación Afín", mouse_callback)
    cv2.imshow("Transformación Afín", image)

    print("Haz click en 3 puntos fuente (verde), luego en 3 puntos destino (rojo).")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Haz click en 3 puntos fuente (verde), luego en 3 puntos destino (rojo).
Selecciona ahora los 3 puntos destino (rojo).
Calculando transformación afín...
Resultado guardado en 'resultado_affine.png'


Calcular la imagen especular a partir de una imagen.

In [ ]:
import cv2

# --- Cargar imagen ---
ruta = "images/cats.png"  # Cambia por tu imagen
img = cv2.imread(ruta)
if img is None:
    raise FileNotFoundError("No se pudo cargar la imagen.")

# --- Espejo horizontal (reflejo vertical) ---
espejo_horizontal = cv2.flip(img, 1)  # 1 = eje Y (horizontal)
# --- Espejo vertical (reflejo horizontal) ---
espejo_vertical = cv2.flip(img, 0)    # 0 = eje X (vertical)
# --- Espejo completo (ejes X y Y) ---
espejo_completo = cv2.flip(img, -1)   # -1 = ambos ejes

# --- Mostrar resultados ---
cv2.imshow("Original", img)
cv2.imshow("Espejo horizontal", espejo_horizontal)
cv2.imshow("Espejo vertical", espejo_vertical)
cv2.imshow("Espejo completo", espejo_completo)

# --- Guardar resultados ---
cv2.imwrite("output/espejo_horizontal.png", espejo_horizontal)
cv2.imwrite("output/espejo_vertical.png", espejo_vertical)
cv2.imwrite("output/espejo_completo.png", espejo_completo)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# --- Cargar imagen ---
ruta = "images/cats.png"  # Cambia por tu imagen
img = cv2.imread(ruta)
if img is None:
    raise FileNotFoundError("No se pudo cargar la imagen.")

# --- Crear imágenes espejadas ---
espejo_horizontal = cv2.flip(img, 1)  # eje Y
espejo_vertical = cv2.flip(img, 0)    # eje X
espejo_completo = cv2.flip(img, -1)   # ambos ejes

# --- Añadir margen y nombre a cada imagen ---
def add_label(im, label, margin=30):
    h, w = im.shape[:2]
    canvas = np.ones((h+margin, w, 3), dtype=np.uint8) * 255  # fondo blanco
    canvas[margin:, :, :] = im
    cv2.putText(canvas, label, (10, margin-10), cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (0, 0, 0), 2, cv2.LINE_AA)
    return canvas

img_labeled = add_label(img, "Original")
hor_labeled = add_label(espejo_horizontal, "Espejo Horizontal")
ver_labeled = add_label(espejo_vertical, "Espejo Vertical")
comp_labeled = add_label(espejo_completo, "Espejo Completo")

# --- Separador entre imágenes ---
sep = 10  # píxeles de margen entre imágenes
horiz_top = np.hstack((img_labeled, np.ones((img_labeled.shape[0], sep, 3), dtype=np.uint8)*255,
                       hor_labeled))
horiz_bottom = np.hstack((ver_labeled, np.ones((ver_labeled.shape[0], sep, 3), dtype=np.uint8)*255,
                          comp_labeled))
grid = np.vstack((horiz_top, np.ones((sep, horiz_top.shape[1], 3), dtype=np.uint8)*255,
                  horiz_bottom))

# --- Mostrar y guardar ---
cv2.imshow("Imagenes Espejo 2x2 con nombres", grid)
cv2.imwrite("output/cuadricula_2x2_nombres.png", grid)

cv2.waitKey(0)
cv2.destroyAllWindows()


Trazar una recta que será el eje de reflexión y “reflejar” la imagen.

In [4]:
import cv2
import numpy as np
import os

# Variables globales
points = []
clone = None
done = False

# Carpeta de salida
os.makedirs("output", exist_ok=True)

def mouse_callback(event, x, y, flags, param):
    global points, clone, done

    if done:
        return

    if event == cv2.EVENT_LBUTTONDOWN:
        points.append([x, y])
        if len(points) == 2:
            # Segundo punto fijado: guardar resultados
            reflected_clean = warp_reflection(clone, points[0], points[1])
            cv2.imwrite("output/resultado_reflexion.png", reflected_clean)

            reflected_with_line = actualizar_reflexion(points[1])
            cv2.imwrite("output/resultado_reflexion_con_linea.png", reflected_with_line)

            print("Guardadas imágenes en output/")
            done = True

    elif event == cv2.EVENT_MOUSEMOVE:
        if len(points) == 1:
            # Mostrar reflexión en tiempo real mientras se mueve el ratón
            display = actualizar_reflexion([x, y])
            cv2.imshow("Reflexion", display)

def actualizar_reflexion(temp_point=None):
    """Actualizar la imagen reflejada en tiempo real."""
    global clone, points

    if len(points) == 0:
        return clone.copy()
    elif len(points) == 1:
        p1 = np.array(points[0], dtype=np.float32)
        if temp_point is None:
            return clone.copy()
        p2 = np.array(temp_point, dtype=np.float32)
    else:
        p1 = np.array(points[0], dtype=np.float32)
        p2 = np.array(points[1], dtype=np.float32)

    h, w = clone.shape[:2]
    M_affine = get_affine_matrix(p1, p2)
    reflected = cv2.warpAffine(clone, M_affine, (w, h))

    # Dibujar línea y puntos temporal
    display = reflected.copy()
    cv2.line(display, tuple(p1.astype(int)), tuple(p2.astype(int)), (255, 0, 0), 2)
    cv2.circle(display, tuple(p1.astype(int)), 5, (0, 0, 255), -1)
    cv2.circle(display, tuple(p2.astype(int)), 5, (0, 0, 255), -1)
    return display

def get_affine_matrix(p1, p2):
    dx, dy = p2 - p1
    ang = np.arctan2(dy, dx)

    T1 = np.array([[1, 0, -p1[0]],
                   [0, 1, -p1[1]],
                   [0, 0, 1]], dtype=np.float32)
    R = np.array([[ np.cos(-ang), -np.sin(-ang), 0],
                  [ np.sin(-ang),  np.cos(-ang), 0],
                  [0, 0, 1]], dtype=np.float32)
    Ref = np.array([[1, 0, 0],
                    [0, -1, 0],
                    [0, 0, 1]], dtype=np.float32)
    R_inv = np.linalg.inv(R)
    T2 = np.array([[1, 0, p1[0]],
                   [0, 1, p1[1]],
                   [0, 0, 1]], dtype=np.float32)
    M = T2 @ R_inv @ Ref @ R @ T1
    return M[:2, :]

def warp_reflection(img, p1, p2):
    h, w = img.shape[:2]
    M_affine = get_affine_matrix(np.array(p1, dtype=np.float32), np.array(p2, dtype=np.float32))
    return cv2.warpAffine(img, M_affine, (w, h))

def main():
    global clone
    ruta = "images/cats.png"
    clone = cv2.imread(ruta)
    if clone is None:
        raise FileNotFoundError("No se pudo cargar la imagen.")

    cv2.namedWindow("Reflexion")
    cv2.setMouseCallback("Reflexion", mouse_callback)
    cv2.imshow("Reflexion", clone)

    print("Haz clic en el primer punto, mueve el ratón para definir la recta, y haz clic en el segundo punto.")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Haz clic en el primer punto, mueve el ratón para definir la recta, y haz clic en el segundo punto.
Guardadas imágenes en output/
